# System Aggregated State Probabilities Large Example

This example demonstrates system-wide aggregated state probabilities in a larger network with multiple classes and stations.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.nc.*
import jline.solvers.mva.*

In [ ]:
// Set verbose level
GlobalConstants.setVerbose(VerboseLevel.STD)

// Create larger network for system aggregated probabilities
val model = Network("largeSysModel")

// Block 1: nodes (larger network)
val node1 = Delay(model, "Delay")
val node2 = Queue(model, "Queue1", SchedStrategy.PS)
val node3 = Queue(model, "Queue2", SchedStrategy.FCFS)
val node4 = Queue(model, "Queue3", SchedStrategy.PS)
val node5 = Queue(model, "Queue4", SchedStrategy.FCFS)
node2.setNumberOfServers(2)
node3.setNumberOfServers(3)
node4.setNumberOfServers(1)
node5.setNumberOfServers(2)

In [ ]:
// Block 2: classes (multiple classes)
val jobclass1 = ClosedClass(model, "Class1", 4, node1, 0)
val jobclass2 = ClosedClass(model, "Class2", 3, node1, 0)
val jobclass3 = ClosedClass(model, "Class3", 2, node1, 0)

In [ ]:
// Set service times
val jobclasses = arrayOf(jobclass1, jobclass2, jobclass3)
val nodes = arrayOf(node1, node2, node3, node4, node5)

// Delay: same for all classes
for (jobclass in jobclasses) {
    node1.setService(jobclass, Exp.fitMean(0.5))
}

// Queue1: different rates
node2.setService(jobclass1, Exp.fitMean(1.0))
node2.setService(jobclass2, Exp.fitMean(1.2))
node2.setService(jobclass3, Exp.fitMean(0.8))

// Queue2: different rates
node3.setService(jobclass1, Exp.fitMean(0.9))
node3.setService(jobclass2, Exp.fitMean(1.1))
node3.setService(jobclass3, Exp.fitMean(1.3))

// Queue3: different rates
node4.setService(jobclass1, Exp.fitMean(1.5))
node4.setService(jobclass2, Exp.fitMean(0.7))
node4.setService(jobclass3, Exp.fitMean(1.0))

// Queue4: different rates
node5.setService(jobclass1, Exp.fitMean(0.6))
node5.setService(jobclass2, Exp.fitMean(1.4))
node5.setService(jobclass3, Exp.fitMean(1.1))

In [ ]:
// Block 3: routing - complex routing patterns
val P = model.initRoutingMatrix()

// Class1: Delay -> Queue1 -> Queue2 -> Queue3 -> Queue4 -> Delay
P.set(jobclass1, jobclass1, node1, node2, 1.0)
P.set(jobclass1, jobclass1, node2, node3, 1.0)
P.set(jobclass1, jobclass1, node3, node4, 1.0)
P.set(jobclass1, jobclass1, node4, node5, 1.0)
P.set(jobclass1, jobclass1, node5, node1, 1.0)

// Class2: Delay -> Queue2 -> Queue1 -> Queue4 -> Queue3 -> Delay
P.set(jobclass2, jobclass2, node1, node3, 1.0)
P.set(jobclass2, jobclass2, node3, node2, 1.0)
P.set(jobclass2, jobclass2, node2, node5, 1.0)
P.set(jobclass2, jobclass2, node5, node4, 1.0)
P.set(jobclass2, jobclass2, node4, node1, 1.0)

// Class3: Delay -> Queue3 -> Queue4 -> Queue1 -> Queue2 -> Delay
P.set(jobclass3, jobclass3, node1, node4, 1.0)
P.set(jobclass3, jobclass3, node4, node5, 1.0)
P.set(jobclass3, jobclass3, node5, node2, 1.0)
P.set(jobclass3, jobclass3, node2, node3, 1.0)
P.set(jobclass3, jobclass3, node3, node1, 1.0)

model.link(P)

In [ ]:
// Solve with NC (network may be too large for CTMC)
println("Solving large network with NC solver:")
try {
    val solverNc = NC(model)
    val avgTable = solverNc.avgTable
    println("NC Results:")
    avgTable.print()
    
    // Test system aggregated probabilities
    println("\nTesting system aggregated probabilities:")
    try {
        val sysProbs = solverNc.probSysAggr
        println("System aggregated probabilities computed successfully")
        println("Number of aggregated states: ${sysProbs.size}")
    } catch (e: Exception) {
        println("System aggregated probabilities not available: ${e.message}")
    }
    
    // Test node-specific probabilities
    node3.setState(intArrayOf(2, 1, 0)) // State [2,1,0] at Queue2
    val nodeProb = solverNc.getProbAggr(node3)
    println("\nProbability of state [2,1,0] at Queue2: $nodeProb")
    
} catch (e: Exception) {
    println("NC solver error: ${e.message}")
}

In [ ]:
// Compare with MVA solution for approximate results
println("\nComparing with MVA solver:")
try {
    val solverMva = MVA(model)
    val avgTableMva = solverMva.avgTable
    println("MVA Results:")
    avgTableMva.print()
} catch (e: Exception) {
    println("MVA solver error: ${e.message}")
}